In [ ]:
!pip3 install mitiq

In [ ]:
from qiskit.circuit.random import random_circuit
from mitiq.interface.mitiq_qiskit import qiskit_utils
from mitiq import PauliString, Observable, zne
from qiskit.providers.aer.noise import NoiseModel
from qiskit import IBMQ, transpile, QuantumCircuit
import numpy as np
import warnings
warnings.filterwarnings("ignore")

In [ ]:
qc = QuantumCircuit(5, 2)

qc.h(0)
qc.cx(0, 1)
qc.cry(np.pi/5, 0, 1)
qc.cx(1,0)
qc.ry(np.pi/4,0)
qc.x(1)
qc.cry(np.pi/3, 1, 0)
qc.cx(0, 1)
qc.h(0)
qc.cx(0, 1)
qc.cry(np.pi/4, 0, 1)
qc.cx(1,0)
qc.ry(np.pi/5,0)
qc.x(1)
qc.cry(np.pi/3, 1, 0)
qc.h(0)
qc.h(1)
#qc.barrier()
qc.measure(range(2), range(2))

qc.draw('mpl')

In [ ]:
obs = Observable(PauliString('ZZZZZ'))
shots = 10000

In [ ]:
def sim_noisy(circ):
    qiskit_utils.execute_with_shots_and_noise(circ, obs.matrix(), NoiseModel.from_backend(backend), shots)
    
def sim_ideal(circ):
    qiskit_utils.execute_with_shots(circ, obs.matrix(), shots)

In [ ]:
if not IBMQ.active_account():
    IBMQ.load_account()
    
provider = IBMQ.get_provider(hub = 'ibm-q', group = 'open', project = 'main')

In [ ]:
backend = provider.get_backend('ibmq_quito')

In [ ]:
mapped_circuit = transpile(qc, backend)

In [ ]:
result_ideal = sim_ideal(mapped_circuit)

In [ ]:
result_noisy = sim_noisy(mapped_circuit)

In [ ]:
result_mitigated = zne.execute_with_zne(mapped_circuit, sim_noisy)

In [ ]:
print('EV unmitigated: ', result_noisy)
print('EV mitigated: ', result_mitigated)
print('EV ideal: ', result_ideal)

In [ ]:
error_unmitigated = abs(result_noisy - result_ideal)
error_unmitigated

In [ ]:
error_mitigated = abs(result_mitigated - result_ideal)
error_mitigated

In [ ]:
error_reduction = (error_unmitigated - error_mitigated) / error_unmitigated
print("Error reduction: ", error_reduction)